# Empathy and Distress Inference with EmotionStack

In [1]:
from transformers import AutoConfig, AutoModelWithHeads
import torch
from utils import utils
from transformers import RobertaTokenizer
import transformers.adapters.composition as ac

In [2]:
""" load data """

train_data, val_data, test_data = utils.load_wassa_dataset()

In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

config = AutoConfig.from_pretrained(
    "roberta-base",
    num_labels=1
)

model = AutoModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:250: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/app/home/alahnala/miniconda3/envs/st/lib/python3.9/site-packages/transformers/adapters/models/roberta.py:228: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a B

In [4]:
""" load adapters """
empathy_adapter_path = "./trained_adapters/EMP_emotion_stack"
distress_adapter_path = "./trained_adapters/DIS_emotion_stack"

empathy_adapter = model.load_adapter(empathy_adapter_path, load_as=empathy_adapter_path.split('/')[-1])
distress_adapter = model.load_adapter(distress_adapter_path, load_as=distress_adapter_path.split('/')[-1])

In [5]:
""" activate """
model.set_active_adapters(ac.Parallel(empathy_adapter, distress_adapter))

In [6]:
def pred_empathy_distress(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokens))
    outputs = model(input_ids)

    emp = outputs[0].logits[0][0].tolist()
    dis = outputs[1].logits[0][0].tolist()
    return emp, dis


def predict_and_print(sentence):
    empathy_score, distress_score = pred_empathy_distress(sentence)
    
    print("Essay: ")
    print(sentence)
    print(f"Empathy: {empathy_score}\tDistress: {distress_score}")
    
    return empathy_score, distress_score

In [7]:
essay = test_data['essay'].values[0]
empathy_score, distress_score = predict_and_print(essay)

Essay: 
Hello Friend, i am writing to you as regards an article i read and i will also like to let you how i felt about the article. I was really sad and gutted by what transpired in the article. It was about an inmate with the name Richardson. Richardson normally stay alone in his cell room, but on this day another inmate was brought to him to start living with Richardson in his cell room the nickname of the Cell room mate was The prophet which has previous record of assaulting about 20 other inmate. This also lead to the assault of Richardson which really mad me sad.
Empathy: 5.9775896072387695	Distress: 5.2304558753967285


In [8]:
essay = test_data['essay'].values[1]
empathy_score, distress_score = predict_and_print(essay)

Essay: 
Hello friend i will like to tell you that India to ratify Paris climate deal in October — India, one of the world's largest greenhouse gas emitters, will ratify the Paris global climate agreement pact next month, Prime Minister Narendra Modi has said. CO2 emissions are believed to be the driving force behind climate change. The Paris deal is the world's first comprehensive climate agreement. It will only come into force legally after it is ratified by at least 55 countries, which between them produce 55% of global carbon emissions.
Empathy: 4.39846658706665	Distress: 5.468973159790039


In [9]:
essay = test_data['essay'].values[2]
empathy_score, distress_score = predict_and_print(essay)

Essay: 
Hello friend I will like to let you know Leonard Cohen Died In His Sleep After A Fall, Manager Says — Songwriter and poet Leonard Cohen died in his sleep after a fall in his Los Angeles home in the middle of the night, his manager has said. “The death was sudden, unexpected, and peaceful,” his manager Robert Kory said in a statement published on the Cohencentric website. Cohen, music’s man of letters whose songs fused religious imagery with themes of redemption and sexual desire, died on Nov. 7, He was 82 when he died.
Empathy: 6.117896556854248	Distress: 4.573461055755615


In [10]:
essay = test_data['essay'].values[3]
empathy_score, distress_score = predict_and_print(essay)

Essay: 
Hello friend, I will like to tell you Qatar loses 3 troops serving with Saudi coalition in Yemen — The Gulf nation of Qatar says three of its soldiers have been killed while participating in Saudi-led military operations in Yemen. It was Qatar's biggest known loss of life in the conflict since its first reported ground deployment there last September. The Qatari military says the deaths occurred on Monday but did not provide any details. The military statement, carried by the official Qatar News Agency, identified the three killed soldiers as Mohammed Awad Salim, Mohammed Dawood Khayal and Mohammed Nasser Mohammed.
Empathy: 5.69073486328125	Distress: 5.085775852203369
